In [1]:
import seaborn as sns
import pandas as pd
import kneed
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from time import time
import numpy as np


In [67]:
df_reduced=pd.read_csv('dielectron_reduced_noise_classes.csv')

In [93]:
df_train=pd.read_csv('train_0.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train_0.csv'

In [ ]:
df_train

In [89]:
df_test=df_reduced.loc[pd.read_csv('test_index_0.csv').iloc[:,1].tolist()]

In [91]:
df_test.head()

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,...,eta2,phi2,Q2,M,M_noise,y_50,y_25,y_10,index_og,index_reduced
0,148029,71813147,20.47820,-13.141200,5.940140,-14.538900,14.42140,-0.887121,2.717060,1,...,-1.25301,0.012442,1,21.4943,21.405405,False,False,False,74881,0
1,146644,966563100,4.84406,-1.700930,4.514950,0.432384,4.82472,0.089499,1.931090,1,...,1.74991,-1.885280,1,10.4335,2.230614,False,False,False,5926,10
2,148031,51673935,4.58313,1.316430,1.613360,4.082790,2.08228,1.425930,0.886402,-1,...,-1.12355,-2.047730,1,28.7693,30.099414,True,False,False,53870,11
3,146644,38246917,4.09694,0.252473,3.339340,2.360080,3.34887,0.656543,1.495330,1,...,-1.08387,-1.465190,-1,18.8874,19.451488,False,False,False,6426,23
4,146944,144383492,9.97302,4.606910,0.553619,8.827860,4.64006,1.399180,0.119598,-1,...,-2.19100,2.884920,1,48.7340,51.789717,True,True,False,93264,31


In [90]:
df_test['index_og']=df_test.iloc[:,2]
df_test['index_reduced']=df_test.iloc[:,1]
df_test=df_test.iloc[:,3:]
df_test=df_test.reset_index(drop=True)

In [92]:
df_test.to_csv('test_index_0.csv')

In [81]:
sample=df_train[df_train['in_sample']==1]

In [82]:
y=sample.y_50
#additional to dropping the endpoints we also drop the run and event numbers, because those aren't predictors
X=sample.iloc[:,1:].drop(columns=['M_noise','M','Run','Event','y_50','y_25','y_10','index_og','index_reduced','in_sample','s1','s2'])

y_test= df_test['y_50']
X_test=df_test.drop(columns=['M_noise','M','Run','Event','y_50','y_25','y_10','index_og','index_reduced'])

In [62]:
X_test.head()

,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2
0,20.47820,-13.141200,5.940140,-14.538900,14.42140,-0.887121,2.717060,1,15.36490,8.115000,0.100974,-13.04670,8.11562,-1.25301,0.012442,1
1,4.84406,-1.700930,4.514950,0.432384,4.82472,0.089499,1.931090,1,9.53545,-0.995149,-3.059380,8.97634,3.21716,1.74991,-1.885280,1
2,4.58313,1.316430,1.613360,4.082790,2.08228,1.425930,0.886402,-1,45.55880,-12.299200,-23.802500,-36.84800,26.79240,-1.12355,-2.047730,1
3,4.09694,0.252473,3.339340,2.360080,3.34887,0.656543,1.495330,1,22.37380,1.431820,-13.507300,-17.77890,13.58300,-1.08387,-1.465190,-1
4,9.97302,4.606910,0.553619,8.827860,4.64006,1.399180,0.119598,-1,60.78550,-12.984600,3.408020,-59.28460,13.42440,-2.19100,2.884920,1


In [63]:
X.head()

,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,in_biased_sample
1149,2.55784,0.201911,2.54447,-0.165691,2.55247,-0.064868,1.49161,1,6.86075,-0.683579,-2.246510,6.446380,2.34821,1.734650,-1.866180,-1,0
1233,5.53466,1.249530,4.71274,2.619400,4.87557,0.514279,1.31162,-1,13.50190,-1.128500,13.424200,0.903938,13.47160,0.067049,1.654660,1,0
1289,3.37171,0.463889,-2.31505,2.407020,2.36107,0.895069,-1.37304,1,121.01200,20.211400,10.319100,-118.865000,22.69330,-2.358060,0.472059,1,0
2002,20.73950,-15.379800,8.12840,11.292400,17.39560,0.610510,2.65540,1,3.11379,2.571660,0.487352,-1.686620,2.61743,-0.606506,0.187287,-1,0
2052,40.30390,-24.555100,-15.19780,28.115400,28.87780,0.862582,-2.58737,1,193.55100,20.608000,30.534400,190.013000,36.83800,2.342970,0.977108,-1,0


In [83]:
rf = RandomForestClassifier()
t0 = time()
rf.fit(X, y)
print ('training time: ', round(time()-t0, 3), 's')

training time:  0.114 s


In [84]:
t1=time()
pred_rf = rf.predict(X)
print ('predicting time: ', round(time()-t1, 3), 's')
print("Accuracy:",metrics.accuracy_score(y, pred_rf))

predicting time:  0.01 s
Accuracy: 1.0


In [85]:
t1=time()
pred_rf = rf.predict(X_test)
print ('predicting time: ', round(time()-t1, 3), 's')
print("Accuracy:",metrics.accuracy_score(y_test, pred_rf))

predicting time:  0.022 s
Accuracy: 0.7575
